In [ ]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor
from heron.models.git_llm.git_gpt_neox import GitGPTNeoXForCausalLM


In [ ]:
device_id = 0

# prepare a pretrained model
model = GitGPTNeoXForCausalLM.from_pretrained(
    'turing-motors/heron-chat-git-ELYZA-fast-7b-v0', torch_dtype=torch.float16
)
model.eval()
model.to(f"cuda:{device_id}")


In [ ]:
# prepare a processor
processor = AutoProcessor.from_pretrained('turing-motors/heron-chat-git-ELYZA-fast-7b-v0')

# prepare inputs
url = "https://www.barnorama.com/wp-content/uploads/2016/12/03-Confusing-Pictures.jpg"
image = Image.open(requests.get(url, stream=True).raw)


In [ ]:
text = f"##human: これは何の写真ですか？\n##gpt: "

# do preprocessing
inputs = processor(
    text,
    image,
    return_tensors="pt",
    truncation=True,
)
inputs = {k: v.to(f"cuda:{device_id}") for k, v in inputs.items()}

# set eos token
eos_token_id_list = [
    processor.tokenizer.pad_token_id,
    processor.tokenizer.eos_token_id,
]


In [ ]:
# do inference
with torch.no_grad():
    out = model.generate(**inputs, max_length=256, do_sample=False, temperature=0., eos_token_id=eos_token_id_list)


In [ ]:
# print result
print(processor.tokenizer.batch_decode(out)[0])
